In [2]:
#import dependencies 
import pandas as pd 
from sqlalchemy import create_engine
import psycopg2
import config 

In [74]:
# read in merged data from repo 
beer_df = pd.read_csv('beer_data.csv')
beer_df.head(2)

,abv,ibu,id,name,style,brewery_id,ounces,brewery_idbrewery_,namebrewery_,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,368,21st Amendment Brewery,San Francisco,CA


In [75]:
#psycopg2 connection to aws through postgres 
conn = psycopg2.connect(database='postgres', user=config.user, password=config.db_password, host=config.host)
cur = conn.cursor()

cur.execute('''CREATE TABLE beer_data ( 
                abv DOUBLE PRECISION, 
                ibu DOUBLE PRECISION, 
                id TEXT, 
                name TEXT, 
                style TEXT, 
                brewery_id TEXT , 
                ounces DOUBLE PRECISION,
                brewery_idbrewery_ INTEGER, 
                namebrewery_ TEXT, 
                city TEXT, 
                state TEXT
           );''')

conn.commit()
conn.close()

In [79]:
#sqlalcehmy database connection
database = f'postgres://{config.user}:{config.db_password}@{config.host}/postgres'
engine = create_engine(database)

In [77]:
# testing querying database 
pd.read_sql_query('SELECT * FROM beer_data' ,con=engine)

,abv,ibu,id,name,style,brewery_id,ounces,brewery_idbrewery_,namebrewery_,city,state


In [80]:
beer_data.to_sql(name='beer_data', if_exists='append', index=False, con=engine, chunksize=500)

In [82]:
# querying first 7 rows 
sample_data = pd.read_sql_query('''SELECT * FROM beer_data 
                                     LIMIT 7;''', con=engine)
sample_data

,abv,ibu,id,name,style,brewery_id,ounces,brewery_idbrewery_,namebrewery_,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,368,21st Amendment Brewery,San Francisco,CA
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
3,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0,368,21st Amendment Brewery,San Francisco,CA
4,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
5,0.049,17.0,801,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
6,0.049,17.0,800,21st Amendment Watermelon Wheat Beer (2006),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
